In [ ]:
import numpy as np
import pandas as pd
pd.set_option('max_columns', None)
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
plt.style.use('ggplot')
import datetime
import lightgbm as lgb
from scipy import stats
from scipy.sparse import hstack, csr_matrix
from sklearn.model_selection import train_test_split, KFold
from wordcloud import WordCloud
from collections import Counter
from nltk.corpus import stopwords
from nltk.util import ngrams
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.preprocessing import StandardScaler
stop = set(stopwords.words('english'))
import os
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls
import xgboost as xgb
import lightgbm as lgb
from sklearn import model_selection
from sklearn.metrics import accuracy_score
import json
import ast
import eli5
import shap
from catboost import CatBoostRegressor
from urllib.request import urlopen
from PIL import Image
from sklearn.preprocessing import LabelEncoder
import time
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn import linear_model

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

        
        
# Any results you write to the current directory are saved as output.

In [ ]:
train = pd.read_csv('../input/tmdb-box-office-prediction/train.csv')
test = pd.read_csv('../input/tmdb-box-office-prediction/test.csv')


In [ ]:
train.info()

In [ ]:
train.head(5)

In [ ]:
# Delete the redundant column which does not add any inormation
train.drop(columns=['homepage', 'imdb_id', 'original_title', 'poster_path'],axis = 1 ,inplace=True)

test.drop(columns=['homepage', 'imdb_id', 'original_title', 'poster_path'],axis = 1 ,inplace=True)

In [ ]:
# number of null values in the data

train.isna().sum()

In [ ]:
# to plot the null values in the different columns
null_values = train.isna().sum().reset_index()


x = null_values.iloc[:,0]
y = null_values.iloc[:,1]

x = x.tolist()
y = y.tolist()

In [ ]:

plt.figure(figsize=(16, 8))
plt.xticks(rotation='vertical')
plt.bar(x, y ,label='Null values count')

In [ ]:
# https://www.kaggle.com/gravix/gradient-in-a-box

dict_columns = ['belongs_to_collection', 'genres', 'production_companies',
                'production_countries', 'spoken_languages', 'Keywords', 'cast', 'crew']

def text_to_dict(df):
    for column in dict_columns:
        df[column] = df[column].apply(lambda x: {} if pd.isna(x) else ast.literal_eval(x) )
    return df
        
train = text_to_dict(train)
test = text_to_dict(test)


In [ ]:
# shape of train and test data set
train.shape, test.shape

In [ ]:
# To find if the film belongs to any big collection
# to do
# normalize


train['has_collection'] = train['belongs_to_collection'].apply(lambda x: len(x) if x != {} else 0)

test['has_collection'] = test['belongs_to_collection'].apply(lambda x: len(x) if x != {} else 0)

train = train.drop(['belongs_to_collection'], axis=1)
test = test.drop(['belongs_to_collection'], axis=1)


# Data processing for the Genre columns

In [ ]:
#from typing import Iterable 
from collections import Iterable


def flatten(items):
    """Yield items from any nested iterable; see Reference."""
    for x in items:
        if isinstance(x, Iterable) and not isinstance(x, (str, bytes)):
            for sub_x in flatten(x):
                yield sub_x
        else:
            yield x
            

In [ ]:
            
Top_genre = Counter(list(flatten(list(train['genres'].apply(lambda x: [i['name'] for i in x] if x != {} else []).values)))).most_common(10)

All_genre = Counter(list(flatten(list(train['genres'].apply(lambda x: [i['name'] for i in x] if x != {} else []).values))))

# to convert the tuple into the dict
dict_top_genre = dict(All_genre)

Total_genres_count = sum(Counter(list(flatten(list(train['genres'].apply(lambda x: [i['name'] for i in x] if x != {} else []).values)))).values())

print(Top_genre)

In [ ]:
train['num_genres'] = train['genres'].apply(lambda x: len(x) if x != {} else 0)
train['all_genres'] = train['genres'].apply(lambda x: ','.join(sorted([i['name'] for i in x])) if x != {} else '')

# max and sum
train['max_genres_norm'] = train['all_genres'].apply(lambda x: max([dict_top_genre[i]/Total_genres_count for i in x.split(',')]) if x != '' else 0)

## create a column based on top genre movies

test['num_genres'] = test['genres'].apply(lambda x: len(x) if x != {} else 0)
test['all_genres'] = test['genres'].apply(lambda x: ','.join(sorted([i['name'] for i in x])) if x != {} else '')

# max and sum
test['max_genres_norm'] = test['all_genres'].apply(lambda x: max([dict_top_genre[i]/Total_genres_count for i in x.split(',')]) if x != '' else 0)


train = train.drop(['genres','all_genres'], axis=1)
test = test.drop(['genres','all_genres'], axis=1)

# Production Companies

#### production compnaies plays a important role for the movie.. often top rated production companies tend to earn good revenue. so parsing the column product companies to get more details from it.

In [ ]:
Top_production_companies = Counter(list(flatten(list(train['production_companies'].apply(lambda x: [i['name'] for i in x] if x != {} else []).values)))).most_common(30)
print(Top_production_companies)

All_production_companies = Counter(list(flatten(list(train['production_companies'].apply(lambda x: [i['name'] for i in x] if x != {} else []).values))))

# to convert the tuple into the dict
dict_top_production_companies = dict(Top_production_companies)

Top_production_companies_count = sum(dict_top_production_companies.values())
print('Total count of the production companies',Top_production_companies_count)


In [ ]:
train['num_companies'] = train['production_companies'].apply(lambda x: len(x) if x != {} else 0)
train['all_production_companies'] = train['production_companies'].apply(lambda x: ','.join(sorted([i['name'] for i in x])) if x != {} else '')

train['max_production_companies_norm'] = train['all_production_companies'].apply(lambda x: max([dict_top_production_companies[i]/Top_production_companies_count if i in dict_top_production_companies.keys() else 0 for i in x.split(',')]) if x != '' else 0)


# for production in Top_production_companies:
#     train['production_company_' + production[0]] = train['all_production_companies'].apply(lambda x: 1 if production[0] in x else 0)

test['num_companies'] = test['production_companies'].apply(lambda x: len(x) if x != {} else 0)
test['all_production_companies'] = test['production_companies'].apply(lambda x: ','.join(sorted([i['name'] for i in x])) if x != {} else '')

test['max_production_companies_norm'] = test['all_production_companies'].apply(lambda x: max([dict_top_production_companies[i]/Top_production_companies_count if i in dict_top_production_companies.keys() else 0 for i in x.split(',')]) if x != '' else 0)


# for production in Top_production_companies:
#     test['production_company_' + production[0]] = test['all_production_companies'].apply(lambda x: 1 if production[0] in x else 0)



train = train.drop(['production_companies', 'all_production_companies'], axis=1)
test = test.drop(['production_companies', 'all_production_companies'], axis=1)

# Production Countries

In [ ]:
list_of_countries = list(train['production_countries'].apply(lambda x: [i['iso_3166_1'] for i in x] if x != {} else []).values)
Top_countries = Counter([i for j in list_of_countries for i in j]).most_common(30)


# to convert the tuple into the dict
dict_top_production_countries = dict(Top_countries)

Top_production_countries_count = sum(dict_top_production_countries.values())
print('Total movie count of the production country',Top_production_countries_count)

In [ ]:
train['num_production_countries'] = train['production_countries'].apply(lambda x: len(x) if x != {} else 0)
train['all_production_countries'] = train['production_countries'].apply(lambda x: ','.join(sorted([i['iso_3166_1'] for i in x])) if x != {} else '')


train['max_production_country_norm'] = train['all_production_countries'].apply(lambda x: max([dict_top_production_countries[i]/Top_production_countries_count if i in dict_top_production_countries.keys() else 0 for i in x.split(',')]) if x != '' else 0)

# for country in Top_countries:
#     train['production_country_' + country[0]] = train['all_countries'].apply(lambda x: 1 if country[0] in x else 0)

test['num_production_countries'] = test['production_countries'].apply(lambda x: len(x) if x != {} else 0)
test['all_production_countries'] = test['production_countries'].apply(lambda x: ','.join(sorted([i['iso_3166_1'] for i in x])) if x != {} else '')


test['max_production_country_norm'] = test['all_production_countries'].apply(lambda x: max([dict_top_production_countries[i]/Top_production_countries_count if i in dict_top_production_countries.keys() else 0 for i in x.split(',')]) if x != '' else 0)



# # for country in Top_countries:
#     test['production_country_' + country[0]] = test['all_countries'].apply(lambda x: 1 if country[0] in x else 0)
    
train = train.drop(['num_production_countries', 'all_production_countries'], axis=1)
test = test.drop(['num_production_countries', 'all_production_countries'], axis=1)

# Spoken language

In [ ]:
## feature engineering part has to take care production country and spoken language

In [ ]:
list_of_languages = list(train['spoken_languages'].apply(lambda x: [i['name'] for i in x] if x != {} else []).values)
top_language = Counter([i for j in list_of_languages for i in j]).most_common(5)

dict_top_language = dict(top_language)

Top_languages_count = sum(dict_top_language.values())
print('Total movie count of the production country',Top_languages_count)

In [ ]:
train['num_languages'] = train['spoken_languages'].apply(lambda x: len(x) if x != {} else 0)
train['all_languages'] = train['spoken_languages'].apply(lambda x: ','.join(sorted([i['name'] for i in x])) if x != {} else '')

train['max_language_norm'] = train['all_languages'].apply(lambda x: max([dict_top_language[i]/Top_languages_count if i in dict_top_language.keys() else 0 for i in x.split(',')]) if x != '' else 0)


# train['max_language_norm'] = train['all_languages'].apply(lambda x: max([dict_top_language[i]/Top_languages_count for i in x.split(',')]) if x != '' else 0)

# top_languages = [m[0] for m in Counter([i for j in list_of_languages for i in j]).most_common(30)]
# for g in top_languages:
#     train['language_' + g] = train['all_languages'].apply(lambda x: 1 if g in x else 0)
    
# test['num_languages'] = test['spoken_languages'].apply(lambda x: len(x) if x != {} else 0)
# test['all_languages'] = test['spoken_languages'].apply(lambda x: ' '.join(sorted([i['name'] for i in x])) if x != {} else '')
# for g in top_languages:
#     test['language_' + g] = test['all_languages'].apply(lambda x: 1 if g in x else 0)

# train = train.drop(['spoken_languages', 'all_languages'], axis=1)
# test = test.drop(['spoken_languages', 'all_languages'], axis=1)

In [ ]:
#cast_name

In [ ]:
list_of_cast_names = list(train['cast'].apply(lambda x: [i['name'] for i in x] if x != {} else []).values)
top_cast_names = Counter([i for j in list_of_cast_names for i in j]).most_common(500)

dict_cast_names = dict(top_cast_names)

Top_cast_names_count = sum(dict_cast_names.values())
print('Total movie count acted wrt cast names',Top_cast_names_count)

In [ ]:
dict_cast_names

In [ ]:
train['num_cast'] = train['cast'].apply(lambda x: len(x) if x != {} else 0)
train['all_cast'] = train['cast'].apply(lambda x: ','.join(sorted([i['name'] for i in x])) if x != {} else '')

train['max_cast_norm'] = train['all_cast'].apply(lambda x: max([dict_cast_names[i]/Top_cast_names_count if i in dict_cast_names.keys() else 0 for i in x.split(',')]) if x != '' else 0)



# Cast Gender

In [ ]:
list_of_cast_genders = list(train['cast'].apply(lambda x: [i['gender'] for i in x] if x != {} else []).values)
Counter([i for j in list_of_cast_genders for i in j])

In [ ]:
train['num_cast'] = train['cast'].apply(lambda x: len(x) if x != {} else 0)

train['genders_0_cast'] = train['cast'].apply(lambda x: sum([1 for i in x if i['gender'] == 0]))
train['genders_1_cast'] = train['cast'].apply(lambda x: sum([1 for i in x if i['gender'] == 1]))
train['genders_2_cast'] = train['cast'].apply(lambda x: sum([1 for i in x if i['gender'] == 2]))


test['num_cast'] = test['cast'].apply(lambda x: len(x) if x != {} else 0)

test['genders_0_cast'] = test['cast'].apply(lambda x: sum([1 for i in x if i['gender'] == 0]))
test['genders_1_cast'] = test['cast'].apply(lambda x: sum([1 for i in x if i['gender'] == 1]))
test['genders_2_cast'] = test['cast'].apply(lambda x: sum([1 for i in x if i['gender'] == 2]))

# train = train.drop(['cast'], axis=1)
# test = test.drop(['cast'], axis=1)

In [ ]:
list_of_crew_departments = list(train['crew'].apply(lambda x: [i['job'] for i in x] if x != {} else []).values)
Counter([i for j in list_of_crew_departments for i in j]).most_common(14)

In [ ]:
# top director 

list_directors_name = list(train['crew'].apply(lambda x: [i['name'] for i in x if i['job'] == 'Director' ] if x != {} else []).values)
dict_director_names = dict(Counter([i for j in list_directors_name for i in j]))

Top_director_names_count = sum(dict_director_names.values())

In [ ]:
# top producer


list_producer_name = list(train['crew'].apply(lambda x: [i['name'] for i in x if i['job'] == 'Producer' ] if x != {} else []).values)
dict_producer_names = dict(Counter([i for j in list_producer_name for i in j]))

Top_producer_names_count = sum(dict_producer_names.values())

In [ ]:
# top Original Music Composer


list_music_composer_name = list(train['crew'].apply(lambda x: [i['name'] for i in x if i['job'] == 'Original Music Composer' ] if x != {} else []).values)
dict_composer_names = dict(Counter([i for j in list_music_composer_name for i in j]).most_common(15))

Top_composer_names_count = sum(dict_composer_names.values())

In [ ]:
train['num_crew'] = train['crew'].apply(lambda x: len(x) if x != {} else 0)

train['crew_director'] = train['crew'].apply(lambda x: ','.join(sorted([i['name'] for i in x if i['job'] == 'Director'])) if x != {} else '')
train['crew_director_norm'] = train['crew_director'].apply(lambda x: [(dict_director_names[i]/Top_director_names_count) if i in dict_director_names.keys() else 0 for i in x.split(',')][0] if x != '' else 0)
train['crew_director_log_norm'] = train['crew_director'].apply(lambda x: [np.log(dict_director_names[i])+1 if i in dict_director_names.keys() else 1 for i in x.split(',')][0] if x != '' else 0)

train['crew_producer'] = train['crew'].apply(lambda x: ','.join(sorted([i['name'] for i in x if i['job'] == 'Producer'])) if x != {} else '')
train['crew_producer_norm'] = train['crew_producer'].apply(lambda x: [(dict_producer_names[i]/Top_producer_names_count) if i in dict_producer_names.keys() else 0 for i in x.split(',')][0] if x != '' else 0)
train['crew_producer_log_norm'] = train['crew_producer'].apply(lambda x: [np.log(dict_producer_names[i])+1 if i in dict_producer_names.keys() else 1 for i in x.split(',')][0] if x != '' else 0)

train['crew_composer'] = train['crew'].apply(lambda x: ','.join(sorted([i['name'] for i in x if i['job'] == 'Original Music Composer'])) if x != {} else '')
train['crew_composer_norm'] = train['crew_composer'].apply(lambda x: [(dict_composer_names[i]/Top_composer_names_count) if i in dict_composer_names.keys() else 0 for i in x.split(',')][0] if x != '' else 0)
train['crew_composer_log_norm'] = train['crew_composer'].apply(lambda x: [np.log(dict_composer_names[i])+1 if i in dict_composer_names.keys() else 1 for i in x.split(',')][0] if x != '' else 0)




#############################################################################################################    

# test['num_crew'] = test['crew'].apply(lambda x: len(x) if x != {} else 0)




# train = train.drop(['crew'], axis=1)
# test = test.drop(['crew'], axis=1)

In [ ]:
from datetime import date
import calendar

train.release_date = pd.to_datetime(train.release_date)
train.loc[:,"Year"] = train["release_date"].dt.year
train.loc[:,"Month"] = train["release_date"].dt.month
train['day_of_week'] = train['release_date'].apply(lambda x: calendar.day_name[x.weekday()])

In [ ]:
train = pd.get_dummies(train, columns=["day_of_week","Month"]).head(3)

In [ ]:
train.head()

# Data Exploration

In [ ]:
fig, ax = plt.subplots(figsize = (16, 6))
plt.subplot(1, 2, 1)
plt.hist(train['revenue']);
plt.title('Distribution of revenue');
plt.subplot(1, 2, 2)
plt.hist(np.log1p(train['revenue']));
plt.title('Distribution of log of revenue');

In [ ]:
train['log_revenue'] = np.log1p(train['revenue'])

In [ ]:
fig, ax = plt.subplots(figsize = (16, 6))
plt.subplot(1, 2, 1)
plt.hist(train['budget']);
plt.title('Distribution of budget');
plt.subplot(1, 2, 2)
plt.hist(np.log1p(train['budget']));
plt.title('Distribution of log of budget');

In [ ]:
train['log_budget'] = np.log1p(train['budget'])
test['log_budget'] = np.log1p(test['budget'])

In [ ]:
train['homepage'].value_counts().head()

In [ ]:
train['has_homepage'] = 0
train.loc[train['homepage'].isnull() == False, 'has_homepage'] = 1
test['has_homepage'] = 0
test.loc[test['homepage'].isnull() == False, 'has_homepage'] = 1

In [ ]:
sns.catplot(x='has_homepage', y='revenue', data=train);
plt.title('Revenue for film with and without homepage');

In [ ]:
plt.figure(figsize=(16, 8))
plt.subplot(1, 2, 1)
sns.boxplot(x='original_language', y='revenue', data=train.loc[train['original_language'].isin(train['original_language'].value_counts().head(10).index)]);
plt.title('Mean revenue per language');
plt.subplot(1, 2, 2)
sns.boxplot(x='original_language', y='log_revenue', data=train.loc[train['original_language'].isin(train['original_language'].value_counts().head(10).index)]);
plt.title('Mean log revenue per language');

In [ ]:
plt.figure(figsize = (12, 12))
text = ' '.join(train['original_title'].values)
wordcloud = WordCloud(max_font_size=None, background_color='white', width=1200, height=1000).generate(text)
plt.imshow(wordcloud)
plt.title('Top words in titles')
plt.axis("off")
plt.show()

In [ ]:
plt.figure(figsize = (12, 12))
text = ' '.join(train['overview'].fillna('').values)
wordcloud = WordCloud(max_font_size=None, background_color='white', width=1200, height=1000).generate(text)
plt.imshow(wordcloud)
plt.title('Top words in overview')
plt.axis("off")
plt.show()

In [ ]:
train['overview']

In [ ]:
vectorizer = TfidfVectorizer(sublinear_tf=True,
            analyzer='word',
            token_pattern=r'\w{1,}',
            ngram_range=(1, 2),
            min_df=5)

overview_text = vectorizer.fit_transform(train['overview'].fillna(''))
linreg = LinearRegression()
linreg.fit(overview_text, train['log_revenue'])
eli5.show_weights(linreg, vec=vectorizer, top=20, feature_filter=lambda x: x != '<BIAS>')

In [ ]:
vectorizer = TfidfVectorizer(
            sublinear_tf=True,
            analyzer='word',
            token_pattern=r'\w{1,}',
            ngram_range=(1, 2),
            min_df=5)

overview_text = vectorizer.fit_transform(train['overview'].fillna(''))
linreg = LinearRegression()
linreg.fit(overview_text, train['log_revenue'])
eli5.show_weights(linreg, vec=vectorizer, top=20, feature_filter=lambda x: x != '<BIAS>')

In [ ]:
print('Target value:', train['log_revenue'][1000])
eli5.show_prediction(linreg, doc=train['overview'].values[1000], vec=vectorizer)

In [ ]:
plt.figure(figsize=(16, 8))
plt.subplot(1, 2, 1)
plt.scatter(train['popularity'], train['revenue'])
plt.title('Revenue vs popularity');
plt.subplot(1, 2, 2)
plt.scatter(train['popularity'], train['log_revenue'])
plt.title('Log Revenue vs popularity');

In [ ]:
test.loc[test['release_date'].isnull() == True, 'release_date'] = '01/01/98'

In [ ]:
def fix_date(x):
    """
    Fixes dates which are in 20xx
    """
    year = x.split('/')[2]
    if int(year) <= 19:
        return x[:-2] + '20' + year
    else:
        return x[:-2] + '19' + year

In [ ]:
train['release_date'] = train['release_date'].apply(lambda x: fix_date(x))
test['release_date'] = test['release_date'].apply(lambda x: fix_date(x))
train['release_date'] = pd.to_datetime(train['release_date'])
test['release_date'] = pd.to_datetime(test['release_date'])

In [ ]:
# creating features based on dates
def process_date(df):
    date_parts = ["year", "weekday", "month", 'weekofyear', 'day', 'quarter']
    for part in date_parts:
        part_col = 'release_date' + "_" + part
        df[part_col] = getattr(df['release_date'].dt, part).astype(int)
    
    return df

train = process_date(train)
test = process_date(test)

In [ ]:
d1 = train['release_date_year'].value_counts().sort_index()
d2 = test['release_date_year'].value_counts().sort_index()
data = [go.Scatter(x=d1.index, y=d1.values, name='train'), go.Scatter(x=d2.index, y=d2.values, name='test')]
layout = go.Layout(dict(title = "Number of films per year",
                  xaxis = dict(title = 'Year'),
                  yaxis = dict(title = 'Count'),
                  ),legend=dict(
                orientation="v"))
py.iplot(dict(data=data, layout=layout))

In [ ]:
d1 = train['release_date_year'].value_counts().sort_index()
d2 = train.groupby(['release_date_year'])['revenue'].sum()
data = [go.Scatter(x=d1.index, y=d1.values, name='film count'), go.Scatter(x=d2.index, y=d2.values, name='total revenue', yaxis='y2')]
layout = go.Layout(dict(title = "Number of films and total revenue per year",
                  xaxis = dict(title = 'Year'),
                  yaxis = dict(title = 'Count'),
                  yaxis2=dict(title='Total revenue', overlaying='y', side='right')
                  ),legend=dict(
                orientation="v"))
py.iplot(dict(data=data, layout=layout))

In [ ]:
d1 = train['release_date_year'].value_counts().sort_index()
d2 = train.groupby(['release_date_year'])['revenue'].mean()
data = [go.Scatter(x=d1.index, y=d1.values, name='film count'), go.Scatter(x=d2.index, y=d2.values, name='mean revenue', yaxis='y2')]
layout = go.Layout(dict(title = "Number of films and average revenue per year",
                  xaxis = dict(title = 'Year'),
                  yaxis = dict(title = 'Count'),
                  yaxis2=dict(title='Average revenue', overlaying='y', side='right')
                  ),legend=dict(
                orientation="v"))
py.iplot(dict(data=data, layout=layout))

In [ ]:
sns.catplot(x='release_date_weekday', y='revenue', data=train);
plt.title('Revenue on different days of week of release');

In [ ]:
plt.figure(figsize=(20, 6))
plt.subplot(1, 3, 1)
plt.hist(train['runtime'].fillna(0) / 60, bins=40);
plt.title('Distribution of length of film in hours');
plt.subplot(1, 3, 2)
plt.scatter(train['runtime'].fillna(0), train['revenue'])
plt.title('runtime vs revenue');
plt.subplot(1, 3, 3)
plt.scatter(train['runtime'].fillna(0), train['popularity'])
plt.title('runtime vs popularity');

In [ ]:
train['status'].value_counts()

In [ ]:
test['status'].value_counts()

In [ ]:
plt.figure(figsize = (12, 12))
text = ' '.join(train['tagline'].fillna('').values)
wordcloud = WordCloud(max_font_size=None, background_color='white', width=1200, height=1000).generate(text)
plt.imshow(wordcloud)
plt.title('Top words in tagline')
plt.axis("off")
plt.show()

In [ ]:
sns.boxplot(x='has_collection', y='revenue', data=train);

In [ ]:
sns.catplot(x='num_genres', y='revenue', data=train);
plt.title('Revenue for different number of genres in the film');

In [ ]:
sns.violinplot(x='genre_Drama', y='revenue', data=train[:100]);

In [ ]:
f, axes = plt.subplots(3, 5, figsize=(24, 12))
plt.suptitle('Violinplot of revenue vs genres')
for i, e in enumerate([col for col in train.columns if 'genre_' in col]):
    sns.violinplot(x=e, y='revenue', data=train, ax=axes[i // 5][i % 5]);

In [ ]:
f, axes = plt.subplots(6, 5, figsize=(24, 32))
plt.suptitle('Violinplot of revenue vs production company')
for i, e in enumerate([col for col in train.columns if 'production_company' in col]):
    sns.violinplot(x=e, y='revenue', data=train, ax=axes[i // 5][i % 5]);

In [ ]:
sns.catplot(x='num_countries', y='revenue', data=train);
plt.title('Revenue for different number of countries producing the film');

In [ ]:
f, axes = plt.subplots(5, 5, figsize=(24, 32))
plt.suptitle('Violinplot of revenue vs production country')
for i, e in enumerate([col for col in train.columns if 'production_country' in col]):
    sns.violinplot(x=e, y='revenue', data=train, ax=axes[i // 5][i % 5]);

In [ ]:
plt.figure(figsize=(16, 8))
plt.subplot(1, 2, 1)
plt.scatter(train['num_cast'], train['revenue'])
plt.title('Number of cast members vs revenue');
plt.subplot(1, 2, 2)
plt.scatter(train['num_cast'], train['log_revenue'])
plt.title('Log Revenue vs number of cast members');

In [ ]:
f, axes = plt.subplots(3, 5, figsize=(24, 18))
plt.suptitle('Violinplot of revenue vs cast')
for i, e in enumerate([col for col in train.columns if 'cast_name' in col]):
    sns.violinplot(x=e, y='revenue', data=train, ax=axes[i // 5][i % 5]);

In [ ]:
f, axes = plt.subplots(3, 5, figsize=(24, 18))
plt.suptitle('Violinplot of revenue vs cast')
for i, e in enumerate([col for col in train.columns if 'cast_character_' in col]):
    sns.violinplot(x=e, y='revenue', data=train, ax=axes[i // 5][i % 5]);

In [ ]:
f, axes = plt.subplots(6, 5, figsize=(24, 32))
plt.suptitle('Violinplot of revenue vs keyword')
for i, e in enumerate([col for col in train.columns if 'keyword_' in col]):
    sns.violinplot(x=e, y='revenue', data=train, ax=axes[i // 5][i % 5]);

In [ ]:
plt.figure(figsize=(16, 8))
plt.subplot(1, 2, 1)
plt.scatter(train['num_crew'], train['revenue'])
plt.title('Number of crew members vs revenue');
plt.subplot(1, 2, 2)
plt.scatter(train['num_crew'], train['log_revenue'])
plt.title('Log Revenue vs number of crew members');

In [ ]:
f, axes = plt.subplots(3, 5, figsize=(24, 18))
plt.suptitle('Violinplot of revenue vs crew_character')
for i, e in enumerate([col for col in train.columns if 'crew_character_' in col]):
    sns.violinplot(x=e, y='revenue', data=train, ax=axes[i // 5][i % 5]);

In [ ]:
f, axes = plt.subplots(3, 5, figsize=(24, 18))
plt.suptitle('Violinplot of revenue vs jobs')
for i, e in enumerate([col for col in train.columns if 'jobs_' in col]):
    sns.violinplot(x=e, y='revenue', data=train, ax=axes[i // 5][i % 5]);

In [ ]:
train = train.drop(['homepage', 'imdb_id', 'poster_path', 'release_date', 'status', 'log_revenue'], axis=1)
test = test.drop(['homepage', 'imdb_id', 'poster_path', 'release_date', 'status'], axis=1)

In [ ]:
for col in train.columns:
    if train[col].nunique() == 1:
        print(col)
        train = train.drop([col], axis=1)
        test = test.drop([col], axis=1)

In [ ]:
for col in ['original_language', 'collection_name']:
    le = LabelEncoder()
    le.fit(list(train[col].fillna('')) + list(test[col].fillna('')))
    train[col] = le.transform(train[col].fillna('').astype(str))
    test[col] = le.transform(test[col].fillna('').astype(str))

In [ ]:
train_texts = train[['title', 'tagline', 'overview', 'original_title']]
test_texts = test[['title', 'tagline', 'overview', 'original_title']]

In [ ]:
for col in ['title', 'tagline', 'overview', 'original_title']:
    train['len_' + col] = train[col].fillna('').apply(lambda x: len(str(x)))
    train['words_' + col] = train[col].fillna('').apply(lambda x: len(str(x.split(' '))))
    train = train.drop(col, axis=1)
    test['len_' + col] = test[col].fillna('').apply(lambda x: len(str(x)))
    test['words_' + col] = test[col].fillna('').apply(lambda x: len(str(x.split(' '))))
    test = test.drop(col, axis=1)

In [ ]:
X = train.drop(['id', 'revenue'], axis=1)
y = np.log1p(train['revenue'])
X_test = test.drop(['id'], axis=1)

In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.1)

In [ ]:
params = {'num_leaves': 30,
         'min_data_in_leaf': 20,
         'objective': 'regression',
         'max_depth': 5,
         'learning_rate': 0.01,
         "boosting": "gbdt",
         "feature_fraction": 0.9,
         "bagging_freq": 1,
         "bagging_fraction": 0.9,
         "bagging_seed": 11,
         "metric": 'rmse',
         "lambda_l1": 0.2,
         "verbosity": -1}
model1 = lgb.LGBMRegressor(**params, n_estimators = 20000, nthread = 4, n_jobs = -1)
model1.fit(X_train, y_train, 
        eval_set=[(X_train, y_train), (X_valid, y_valid)], eval_metric='rmse',
        verbose=1000, early_stopping_rounds=200)

In [ ]:
eli5.show_weights(model1, feature_filter=lambda x: x != '<BIAS>')